In [90]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/BigData

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/BigData


In [91]:
#import
import pandas as pd
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE, Isomap
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, Normalizer
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, mean_absolute_error
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, train_test_split, KFold
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputRegressor
import random

In [92]:
df = pd.read_csv('./results.csv')
#date 형식 변환
df['date'] = [i.replace('-','') for i in df['date']]

df_old =  pd.read_csv('./results.csv')

In [ ]:
#all_list : tournament, country, city, home_team, away_team
all_list = ((df['tournament'].unique()).tolist() + (df['country'].unique()).tolist() + (df['city'].unique()).tolist() + (df['home_team'].unique()).tolist() + (df['away_team'].unique()).tolist())

#all_list를 왜 nat_dic으로 변환할까?
uni_len = '***'.join(all_list)
nat_enc = tf.keras.preprocessing.text.Tokenizer(num_words=len(all_list),
    filters='',
    lower=False,
    split = '***')
nat_enc.fit_on_texts([uni_len])
nat_dic = nat_enc.word_index

In [93]:
#값을 enumerate로 변환
for step, val in enumerate(df['tournament']):
    df['tournament'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['city']):
    df['city'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['country']):
    df['country'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['home_team']):
    df['home_team'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['away_team']):
    df['away_team'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['neutral']):
    if df['neutral'].iloc[step] == False:
        df['neutral'].iloc[step] = 0
    if df['neutral'].iloc[step] == True:
        df['neutral'].iloc[step] = 1

In [ ]:
#df_new
df_new = df
#len의 역할?
temp_tor = []
for i in df_old['tournament']:
    temp_tor.append(len(i))
temp_away = []
for i in df_old['away_team']:
    temp_away.append(len(i))
temp_home = []
for i in df_old['home_team']:
    temp_home.append(len(i))
df_new['tor_len'] = temp_tor
df_new['home_len'] = temp_home
df_new['away_len'] = temp_away

#df_new : date 형태 변환 ex. 18721130 -> -4.870880
exp = df_new['date'].to_numpy()
scaler = StandardScaler()
exp_2 = scaler.fit_transform(exp.reshape(-1,1))
df_new['date'] = (np.ravel(exp_2)).tolist()
# 결측값 있는 행 제거
df_new = df_new.dropna()

df_label1 = df_new['home_score']
df_label2 = df_new['away_score']
df_new = df_new.drop(['home_score', 'away_score'], axis = 1)


In [ ]:
x_train = df_new.to_numpy()

In [ ]:
#df_label
df_train = pd.DataFrame(data=x_train, columns=df_new.columns)
df_label = pd.concat([df_label1, df_label2],axis = 1)
df_label.columns = ['home_score', 'away_score']
#win column 추가
win = []
for h, a in zip(df_label['home_score'], df_label['away_score']):
    if h > a:
        win.append('home_win')
    if a > h:
        win.append('away_win')
    if a == h:
        win.append('draw')
df_label['win'] = win


In [ ]:
df_label

,home_score,away_score,win
0,0.0,0.0,draw
1,4.0,2.0,home_win
2,2.0,1.0,home_win
3,2.0,2.0,draw
4,3.0,0.0,home_win
...,...,...,...
44099,2.0,1.0,home_win
44100,0.0,5.0,away_win
44101,2.0,0.0,home_win
44102,0.0,0.0,draw


In [ ]:
#x_train = df_new.to_numpy()
Norm = Normalizer()
x_temp = Norm.fit_transform(x_train)
df_temp = pd.DataFrame(data=x_temp, columns=df_new.columns)

In [ ]:
df_corr = df_temp.corr()

In [ ]:
"""
sns.set_style('darkgrid')

fig, axs = plt.subplots(2,3, figsize=(20,10))

axs[0,0].scatter(x = df_train['home_len'], y = df_label1, c='blue', marker='.', alpha=0.7)
axs[0,0].scatter(x= df_train['away_len'], y = df_label2, c='red', marker='1', alpha = 0.5)
axs[0,0].legend(['home_score', 'away_score'])
axs[0,0].set_xlabel('Team( home / away )_name_length')
axs[0,0].set_ylabel('Goals_scored')
axs[0,0].set_title('Goal vs Team_name_length')

axs[1,0].set_title('Mean_goal_scored_in_each_comp_length')
sns.barplot(x=df_train['tor_len'], y=df_label['home_score'], ax=axs[1,0]);
sns.barplot(x = df_train['tor_len'], y=df_label['away_score'], ax = axs[1,0]);
axs[1,0].set_ylabel('home/away_scores')

axs[0,1].set_title('Total_goal_scored v/s date')
axs[0,1].scatter(df_train['date'],df_label['home_score'], c='blue', alpha=0.5, marker = '.');
axs[0,1].scatter(df_train['date'],df_label['away_score'], c='red', alpha=0.5, marker = '.');
axs[0,1].set_xlabel('Scaled_date')
axs[0,1].set_ylabel('total_goals_scored')
axs[0,1].legend(['home_score', 'away_score'])

axs[0,2].set_title('Mean_goal_scored_in_each_in_each_results')
sns.barplot(x=df_label['win'], y = df_label['home_score'], ax=axs[0,2]);
sns.barplot(x = df_label['win'], y = df_label['away_score'], ax = axs[0,2]);
axs[0,2].set_ylabel('home/away_scores')
axs[0,2].set_xlabel('result')

axs[1,1].set_title('Total_goals_scored v/s results')
sns.barplot(x=df_label['win'], y=df_label['home_score']+df_label['away_score'], hue=df_train['neutral'], ax=axs[1,1]);
axs[1,1].set_ylabel('Total_goals_scored')

axs[1,2].set_title('Correlation - Matrix')
sns.heatmap(data = df_corr, square=True, annot=True, ax=axs[1,2])


plt.tight_layout(pad=2);
"""

"\nsns.set_style('darkgrid')\n\nfig, axs = plt.subplots(2,3, figsize=(20,10))\n\naxs[0,0].scatter(x = df_train['home_len'], y = df_label1, c='blue', marker='.', alpha=0.7)\naxs[0,0].scatter(x= df_train['away_len'], y = df_label2, c='red', marker='1', alpha = 0.5)\naxs[0,0].legend(['home_score', 'away_score'])\naxs[0,0].set_xlabel('Team( home / away )_name_length')\naxs[0,0].set_ylabel('Goals_scored')\naxs[0,0].set_title('Goal vs Team_name_length')\n\naxs[1,0].set_title('Mean_goal_scored_in_each_comp_length')\nsns.barplot(x=df_train['tor_len'], y=df_label['home_score'], ax=axs[1,0]);\nsns.barplot(x = df_train['tor_len'], y=df_label['away_score'], ax = axs[1,0]);\naxs[1,0].set_ylabel('home/away_scores')\n\naxs[0,1].set_title('Total_goal_scored v/s date')\naxs[0,1].scatter(df_train['date'],df_label['home_score'], c='blue', alpha=0.5, marker = '.');\naxs[0,1].scatter(df_train['date'],df_label['away_score'], c='red', alpha=0.5, marker = '.');\naxs[0,1].set_xlabel('Scaled_date')\naxs[0,1].se

In [ ]:
df_label.drop('win', axis=1)

,home_score,away_score
0,0.0,0.0
1,4.0,2.0
2,2.0,1.0
3,2.0,2.0
4,3.0,0.0
...,...,...
44099,2.0,1.0
44100,0.0,5.0
44101,2.0,0.0
44102,0.0,0.0


In [ ]:
df_temp.drop('city', axis=1)

,date,home_team,away_team,tournament,country,neutral,tor_len,home_len,away_len
0,-0.008538,0.061351,0.063104,0.534631,0.061351,0.0,0.014023,0.014023,0.012270
1,-0.008459,0.063005,0.061254,0.533789,0.063005,0.0,0.014001,0.012251,0.014001
2,-0.008401,0.061351,0.063104,0.534631,0.061351,0.0,0.014023,0.014023,0.012270
3,-0.008317,0.063005,0.061255,0.533789,0.063005,0.0,0.014001,0.012251,0.014001
4,-0.008259,0.061351,0.063104,0.534632,0.061351,0.0,0.014023,0.014023,0.012270
...,...,...,...,...,...,...,...,...,...
44099,0.000498,0.029217,0.014203,0.123765,0.029217,0.0,0.003246,0.002435,0.003246
44100,0.001077,0.152813,0.036886,0.267862,0.152813,0.0,0.007026,0.017565,0.007904
44101,0.000851,0.135273,0.133885,0.211581,0.135273,0.0,0.005550,0.006937,0.006937
44102,0.001753,0.101466,0.141481,0.451597,0.101466,0.0,0.014291,0.012862,0.010004


In [ ]:
y_es = df_label.drop('win', axis=1).to_numpy()
x_es = df_temp.drop('city', axis=1).to_numpy()
x_t, x_val , y_t, y_val = train_test_split(x_es, y_es, random_state=28)

In [ ]:
#x_t=pd.DataFrame(data=x_temp)
#x_t

In [ ]:
params = {
    "estimator__colsample_bylevel": uniform(0.7, 0.3),
    "estimator__gamma": uniform(0, 0.5),
    "estimator__learning_rate": uniform(0.003, 0.3),
    "estimator__max_depth": randint(2, 6),
    "estimator__n_estimators": randint(100, 500), 
    "estimator__subsample": uniform(0.6, 0.4)
}

In [ ]:
#Multioutput regression are regression problems that involve predicting two or more numerical values given an input example.
xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', eval_metric=['mae'], random_state = 1, use_label_encoder=False))

In [ ]:
#search : 하이퍼 파라미터 튜닝
search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state =1, n_iter=5, cv=3,
                            verbose=0, n_jobs=1, return_train_score=True) 
search.fit(x_t, y_t) 

best_param = search.best_params_
print('-')
print(search.best_score_)
print(best_param)
print('-')

-
0.10852745493991223
{'estimator__colsample_bylevel': 0.7277015784306393, 'estimator__gamma': 0.09313010568883545, 'estimator__learning_rate': 0.10666821811291433, 'estimator__max_depth': 4, 'estimator__n_estimators': 381, 'estimator__subsample': 0.8155266936013428}
-


In [ ]:
#RandomizedSearchCV로 찾은 best param을 모델에 적용
xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror',eval_metric=['mae'], random_state = 1, 
                              use_label_encoder=False, **best_param)).fit(x_t, y_t) 

In [ ]:
#MAE : MSE와 유사한 loss function
#이 코드에서 loss function이 적용되는 부분이 어디지? 못 찾았음
print('MAE: ',mean_absolute_error(y_val, xgb_model.predict(x_val)))

MAE:  1.0839428344574897


In [ ]:
#K-fold 교차 검증:
#전체 훈련 데이터를 K개의 그룹으로 묶고, 그중 한 그룹을 검증셋으로 사용합니다. 
#이때 각각의 모든 데이터 그룹이 적어도 한 번씩은 검증셋으로 사용되어야 합니다. 
#K-fold 교차 검증을 사용함으로써 검정 데이터에 모델이 과적합 되는 것을 막을 수 있습니다. 
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
scores = cross_val_score(xgb_model, x_val, y_val, scoring='neg_mean_absolute_error', cv=folds, verbose=0)
print('KFold_Neg_MAE: ',np.mean(scores))

KFold_Neg_MAE:  -1.0911797128265908


In [ ]:
#inp_list: 사용자가 입력한 home_team, away team
def predict_score(inp_list, autofill, Norm, seed, model):
    ind_list = []
    for i in inp_list:
        ind_list.append(nat_dic.get(i))
    seed[:,1] = ind_list[0]
    seed[:,2] = ind_list[1]
    x_test = Norm.fit_transform(seed)
    pred = model.predict(x_test)
    return np.ravel(pred).tolist()

In [ ]:
inp = str(input("Enter Home & Away team (like Home Away): "))
autofill = True
inp = inp.split(' ')

Enter Home & Away team (like Home Away): a,b


In [ ]:
#seed의 역할
seed = x_train[random.randint(0,x_train.shape[0])].reshape(1,-1)
seed = np.delete(seed, 4, axis=1)

In [ ]:
params = {
    'autofill': True,
    'Norm': Norm,
    'seed': seed,
    'model': xgb_model
}
ans = predict_score(inp, **params)